In [1]:
from skimage.exposure import rescale_intensity
import numpy as np
import cv2

In [20]:
IMAGE_PATH = 'jemma.png'

In [3]:
def convolve(image, K):
    # Grab the spatial dimensions of the image and kernel
    (iH, iW) = image.shape[:2]
    (kH, kW) = K.shape[:2]
    
    # Allocate memory for output image take care to pad the orders of the
    # input image so the spatial size ie width and height, are not reduced
    pad = (kW - 1) // 2
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    output = np.zeros((iH, iW), dtype="float")
    
    # Loop over the input image, sliding the kernel across each (x, y)-coordinate from 
    # Left-2-Right and Top-2-Bottom
    for y in np.arange(pad, iH + pad):
        for x in np.arange(pad, iW + pad):
            # Extract the ROI of the image by extracting the *center*
            # region of the current (x, y)-coordinates dimensions
            roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
            
            # Perform the actual convolution by taking the element-wise multiplications 
            # between the ROI and the kernel, the summing the matrix
            k = (roi * K).sum()
            
            # Store the convolved value in the output (x, y)-coordinate of the
            # output image
            output[y - pad, x - pad] = k
            
    # Rescale the output image to be in the range [0, 255]
    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype("uint8")
    
    return output

In [4]:
# Construct the average blurring kernels used to smooth an image
smallBlur = np.ones((7, 7), dtype="float") * (1.0 / (7 * 7))
largeBlur = np.ones((21, 21), dtype="float") * (1.0 / (21 * 21))

In [5]:
# Construct a a sharpening filter
sharpen = np.array((
    [0, -1, 0], [-1, 5, -1], [0, -1, 0]
), dtype="int")

# Laplacian Kernel for edge detection
laplacian = np.array((
    [0, -1, 0], [-1, 5, -1], [0, -1, 0]
), dtype="int")

# Sobel X-axis kernel
sobelX = np.array((
    [-1, 0, 1], [-2, 0, 2], [-1, 0, 1]
), dtype="int")

# Sobel Y-axis kernel
sobelY = np.array((
    [-1, -2, -1], [0, 0, 0], [1, 2, 1]
), dtype="int")

# Emboss kernel
emboss = np.array((
    [-2, -1, 0], [-1, 1, 1], [0, 1, 2]
), dtype="int")

In [6]:
# Construct the kernel bank, a list of kernels we are going to apply using both our 
# custom 'convolve' funcitona and opencvs filter2D

kernelBank = (
    ("small_blur", smallBlur),
    ("largeBlur", largeBlur),
    ("sharpen", sharpen),
    ("laplacian", laplacian),
    ("sobelX", sobelX),
    ("sobelY", sobelY),
    ("emboss", emboss)
)

In [22]:
# Load the input image and conver it to grayscale 
gray = cv2.imread(IMAGE_PATH, 0)
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


In [24]:
# Loop over kernels 
for (kernelName, K) in kernelBank:
    # Apply the kernel to the grayscale image using both our custom
    # 'convolve' function and openCVs filter2D function
    print("[INFO] applying {} kernel".format(kernelName))
    convolveOutput = convolve(gray, K)
    openCVOutput = cv2.filter2D(gray, -1, K)
    
    # Show the output images
    cv2.imshow("Original", gray)
    cv2.imshow("{} - convolve".format(kernelName), convolveOutput)
    cv2.imshow("{} - opencv".format(kernelName), openCVOutput)
    

[INFO] applying small_blur kernel
[INFO] applying largeBlur kernel
[INFO] applying sharpen kernel
[INFO] applying laplacian kernel
[INFO] applying sobelX kernel
[INFO] applying sobelY kernel
[INFO] applying emboss kernel
